In [1]:
import numpy as np
import cv2
import math
import time
import struct
from pynq import DefaultIP
from pynq import Overlay

In [4]:
from __future__ import print_function

import numpy as np
import cv2 as cv

ply_header = '''ply
format ascii 1.0
element vertex %(vert_num)d
property float x
property float y
property float z
property uchar red
property uchar green
property uchar blue
end_header
'''

def write_ply(fn, verts, colors):
    verts = verts.reshape(-1, 3)
    colors = colors.reshape(-1, 3)
    verts = np.hstack([verts, colors])
    with open(fn, 'wb') as f:
        f.write((ply_header % dict(vert_num=len(verts))).encode('utf-8'))
        np.savetxt(f, verts, fmt='%f %f %f %d %d %d ')


def main():
    print('loading images...')
    imgL = cv.pyrDown(cv.imread('aloeL.png'))  # downscale images for faster processing
    imgR = cv.pyrDown(cv.imread('aloeR.png'))
    
    # disparity range is tuned for 'aloe' image pair
    window_size = 3
    min_disp = 16
    num_disp = 112-min_disp
    stereo = cv.StereoSGBM_create(minDisparity = min_disp,
        numDisparities = num_disp,
        blockSize = 16,
        P1 = 8*3*window_size**2,
        P2 = 32*3*window_size**2,
        disp12MaxDiff = 1,
        uniquenessRatio = 10,
        speckleWindowSize = 100,
        speckleRange = 32
    )
    
    start_time_disp = time.clock()
    print('computing disparity...')
    disp = stereo.compute(imgL, imgR).astype(np.float32) / 16.0
    end_time_disp = time.clock()
    elapsed_time_disp = end_time_disp-start_time_disp
    print("Start time: disparity ",start_time_disp)
    print("End time:  disparity",end_time_disp)
    print("Elapsed time: disparity ",elapsed_time_disp)

    print('generating 3d point cloud...',)
    start_time_Cloud = time.clock()

    h, w = imgL.shape[:2]
    f = 0.8*w                        # guess for focal length
    Q = np.float32([[1, 0, 0, -0.5*w],
                    [0,-1, 0,  0.5*h], # turn points 180 deg around x-axis,
                    [0, 0, 0,     -f], # so that y-axis looks up
                    [0, 0, 1,      0]])
    points = cv.reprojectImageTo3D(disp, Q)
    colors = cv.cvtColor(imgL, cv.COLOR_BGR2RGB)
    mask = disp > disp.min()
    out_points = points[mask]
    out_colors = colors[mask]
    
    end_time_Cloud = time.clock()
    elapsed_time_Cloud = end_time_Cloud-start_time_Cloud

    print("Start time: 3d point cloud ",start_time_disp)
    print("End time:  3d point cloud",end_time_disp)
    print("Elapsed time: 3d point cloud ",elapsed_time_Cloud)
    return out_points, out_colors
    print(np.shape(cube))

if __name__ == '__main__':
    print(__doc__)
    cube,colors = main()
    print('Done')
# Save 3D point cloud as Cube
np.save("colors.npy",colors)
np.save("cube.npy",cube)

Automatically created module for IPython interactive environment
loading images...
computing disparity...
Start time: disparity  42.973997
End time:  disparity 55.505393
Elapsed time: disparity  12.531396
generating 3d point cloud...
Start time: 3d point cloud  42.973997
End time:  3d point cloud 55.505393
Elapsed time: 3d point cloud  0.2616530000000026
Done


In [ ]:
seconds = time.clock()
seconds

In [ ]:
znear = 10
zfar = 1000
FOV = 1.5708 # radians
a = 1
f = math.tan(FOV*0.5)

In [ ]:
def mat_calc(x,y,z,matrix1,matrix2,matrix3,matrix4,matrix5,matrix6,matrix7,matrix8,matrix9): 
    out_x = x*matrix1 + y*matrix2 + z*matrix3 
    out_y = x*matrix4 + y*matrix5 + z*matrix6 
    out_z = x*matrix7 + y*matrix8 + z*matrix9 
        
    return out_x,out_y,out_z

In [ ]:
overlay = Overlay('mat_calc.bit')
overlay?

In [ ]:
mat_calc_ip = overlay.mat_calc
mat_calc_ip?

In [ ]:
class AddDriver(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)

    bindto = ['xilinx.com:hls:mat_calc:1.0']

    def mat_calc(self, x, y, z,mat_0_0,mat_0_1,mat_0_2,mat_1_0,mat_1_1,mat_1_2,mat_2_0,mat_2_1,mat_2_2):
        self.write(0x10, x)
        self.write(0x18, y)
        self.write(0x20, z)
        self.write(0x28, mat_0_0)
        self.write(0x30, mat_0_1)
        self.write(0x38, mat_0_2)
        self.write(0x40, mat_1_0)
        self.write(0x48, mat_1_1)
        self.write(0x50, mat_1_2)
        self.write(0x58, mat_2_0)
        self.write(0x60, mat_2_1)
        self.write(0x68, mat_2_2)
        
        return self.read(0x70),self.read(0x78),self.read(0x80)
#         return self.read(0x78)
#         return self.read(0x80)
    
overlay = Overlay('mat_calc.bit')

In [ ]:
# colors = np.load("colors.npy")
# cube = np.load("cube.npy")

In [ ]:
print("Min",np.min(cube[:,0]))
cube_0_min = np.min(cube[:,0])
print("Max",np.max(cube[:,0]))
cube_0_max = np.max(cube[:,0]) + cube_0_min

print("Min",np.min(cube[:,1]))
cube_1_min = np.min(cube[:,1])
print("Max",np.max(cube[:,1]))
cube_1_max = np.max(cube[:,1])+cube_1_min

print("Min",np.min(cube[:,2]))
cube_2_min = np.min(cube[:,2])
print("Max",np.max(cube[:,2]))
cube_2_max = np.max(cube[:,2])+cube_2_min

In [ ]:
# Makes range 0->1
for i in range(len(cube)):
    cube[i,0] = (cube[i,0]+cube_0_min)/cube_0_max
    cube[i,1] = (cube[i,1]+cube_1_min)/cube_1_max
    cube[i,2] = (cube[i,2]+cube_2_min)/cube_2_max

In [ ]:
for i in range(len(cube)):
    cube[i,0] = int(cube[i,0]*1000)
    cube[i,1] = int(cube[i,1]*1000)
    cube[i,2] = int(cube[i,2]*1000)

In [ ]:
a = 800/500
f = 1/math.tan(FOV/2)
q = (zfar)/(zfar-znear)
theta = 1.5
z_rot = np.array([[int(1000*math.cos(theta)),int(1000*math.sin(theta)),0],
                     [int(1000*-math.sin(theta)),int(1000*math.cos(theta)),0],
                    [0, 0, 1]])

#normalised = [aspect_ratio*scaling_factor*x, scaling_factor*y, z*ztrans]

matrix = np.array([[int(1000*a*f), 0, 0],
                  [0, int(1000*f), 0],
                  [0, 0, int(1000*q)]])


#print(type(z_rot[0,0]))
out_x2,out_y2,out_z2= mat_calc(int(cube[i,0]),int(cube[i,1]),int(cube[i,2]),int(z_rot[0,0]),int(z_rot[0,1]),int(z_rot[0,2]),int(z_rot[1,0]),int(z_rot[1,1]),int(z_rot[1,2]),int(z_rot[2,0]),int(z_rot[2,1]),int(z_rot[2,2]))
out_x1,out_y1,out_z1= mat_calc(cube[i,0],cube[i,1],cube[i,2],z_rot[0,0],z_rot[0,1],z_rot[0,2],z_rot[1,0],z_rot[1,1],z_rot[1,2],z_rot[2,0],z_rot[2,1],z_rot[2,2])
print("Sample 3D point",int(cube[i,0]),int(cube[i,1]),int(cube[i,2]),int(z_rot[0,0]),int(z_rot[0,1]),int(z_rot[0,2]),int(z_rot[1,0]),int(z_rot[1,1]),int(z_rot[1,2]),int(z_rot[2,0]),int(z_rot[2,1]),int(z_rot[2,2]))
print("2D Projection By Board",out_x1,out_y1,out_z1)

# print("Int with Manual",out_x2,out_y2,out_z2)

out_xt,out_yt,out_zt= overlay.mat_calc.mat_calc(4294966373, 4294947544, 1110, 70, 997, 0, 4294966299, 70, 0, 0, 0, 1)
out_x,out_y,out_z= overlay.mat_calc.mat_calc(int(cube[i,0]),int(cube[i,1]),int(cube[i,2]),int(z_rot[0,0]),int(z_rot[0,1]),int(z_rot[0,2]),int(z_rot[1,0]),int(z_rot[1,1]),int(z_rot[1,2]),int(z_rot[2,0]),int(z_rot[2,1]),int(z_rot[2,2]))


if(out_x>500000000):
    out_x = -1*(4294967296 - out_x)
    
if(out_y>500000000):
    out_y = -1*(4294967296 - out_y)
    

print("2D projection using accelerator",out_x,out_y,out_z)
# print("Acc",out_xt,out_yt,out_zt)

In [ ]:
# with overlay
start_time = time.clock()
num_epochs = 1
iter = int(len(cube)/100)
proj = np.zeros((num_epochs,iter,3))
for epoch in range(num_epochs):
    theta = 0+0.1*time.clock()
    #z_rot = np.array([[math.cos(4.71238898038 ),math.sin(4.71238898038),0],
     #           [-math.sin(4.71238898038),math.cos(4.71238898038),0],
      #          [0,0,1]])
    z_rot = np.array([[int(1000*math.cos(theta)),int(1000*math.sin(theta)),0],
                     [int(1000*-math.sin(theta)),int(1000*math.cos(theta)),0],
                    [0, 0, 1]])
    x_rot = np.array([[1,0,0],
                  [0,math.cos(theta),-math.sin(theta)],
                  [0,math.sin(theta),math.cos(theta)]])

    y_rot = np.array([[math.cos(theta),0,math.sin(theta)],
                  [0,1,0],
                  [-math.sin(theta),0,math.cos(theta)]])


    cube_rot = np.zeros((iter,3))
    
    rot_operation_start = time.clock()
    
    for i in range(iter):
        rot_start = time.clock()
        
        cube_rot[i,0],cube_rot[i,1],cube_rot[i,2] = overlay.mat_calc.mat_calc(int(cube[i,0]),int(cube[i,1]),int(cube[i,2]),int(z_rot[0,0]),int(z_rot[0,1]),int(z_rot[0,2]),int(z_rot[1,0]),int(z_rot[1,1]),int(z_rot[1,2]),int(z_rot[2,0]),int(z_rot[2,1]),int(z_rot[2,2]))
#         cube_rot[i,0],cube_rot[i,1],cube_rot[i,2] = mat_calc(int(cube[i,0]),int(cube[i,1]),int(cube[i,2]),int(z_rot[0,0]),int(z_rot[0,1]),int(z_rot[0,2]),int(z_rot[1,0]),int(z_rot[1,1]),int(z_rot[1,2]),int(z_rot[2,0]),int(z_rot[2,1]),int(z_rot[2,2]))

    # Constrain +ve -ve
        if(cube_rot[i,0]>500000000):
            cube_rot[i,0] = -1*(4294967296 - cube_rot[i,0])
    
        if(cube_rot[i,1]>500000000):
            cube_rot[i,1] = -1*(4294967296 - cube_rot[i,1])
        #cube_rot[i,0],cube_rot[i,1],cube_rot[i,2] = mat_calc(cube[i,0],cube[i,1],cube[i,2],x_rot)
        #pass
        cube_rot[i,0] = cube_rot[i,0]/1000
        cube_rot[i,1] = cube_rot[i,1]/1000
        cube_rot[i,2] = cube_rot[i,2]/1000
        rot_end = time.clock()
        rot_elapsed = rot_end - rot_start
        print("Rotation: ",i, rot_elapsed)
    
    rot_operation_end = time.clock()
    rot_operation_elapsed = rot_operation_end - rot_operation_start
    print("Rotated",epoch, rot_operation_elapsed)
    
    cube_trans = cube_rot
    #cube_trans = cube
    
    trans_operation_start = time.clock()
    for i in range(iter):
        trans_start = time.clock()
        cube_trans[i,2] = cube[i,2] + 3
        proj[epoch,i] = overlay.mat_calc.mat_calc(int(cube_trans[i,0]),int(cube_trans[i,1]),int(cube_trans[i,2]),int(matrix[0,0]),int(matrix[0,1]),int(matrix[0,2]),int(matrix[1,0]),int(matrix[1,1]),int(matrix[1,2]),int(matrix[2,0]),int(matrix[2,1]),int(matrix[2,2]))
#         proj[epoch,i] = mat_calc(int(cube_trans[i,0]),int(cube_trans[i,1]),int(cube_trans[i,2]),int(matrix[0,0]),int(matrix[0,1]),int(matrix[0,2]),int(matrix[1,0]),int(matrix[1,1]),int(matrix[1,2]),int(matrix[2,0]),int(matrix[2,1]),int(matrix[2,2]))
    
    #Constrain +ve -ve here
    
        if(proj[epoch,i,0]>500000000):
            proj[epoch,i,0] = -1*(4294967296 - proj[epoch,i,0])
    
        if(proj[epoch,i,1]>500000000):
            proj[epoch,i,1] = -1*(4294967296 - proj[epoch,i,1])
    
        proj[epoch,i,0] = proj[epoch,i,0]/500000
        proj[epoch,i,1] = proj[epoch,i,1]/1000000
        proj[epoch,i,2] = proj[epoch,i,2]/1000000
        
        trans_end = time.clock()
        trans_elapsed = time.clock()
        print("Translated: ",i,trans_elapsed)
    
    trans_operation_end = time.clock()
    trans_operation_elapsed = trans_operation_end - trans_operation_start
    print("Translated",epoch, trans_operation_elapsed)
    
    adj_operation_start = time.clock()
    for i in range(iter):
        proj[epoch,i,0] = 10*(proj[epoch,i,0]+10)
        proj[epoch,i,1] = 20*(proj[epoch,i,1]+15)
    
    adj_operation_end = time.clock()
    adj_operation_elapsed = trans_operation_end - trans_operation_start
    print("Adjusted",epoch, adj_operation_elapsed)
    
np.save("proj.npy",proj)
np.save("cube.npy",cube)
  
end_time = time.clock()
elapsed_time = end_time-start_time
print("Using overlay ")
print("Start time: ",start_time)
print("End time: ",end_time)
print("Elapsed time: ",elapsed_time)

In [ ]:
# without overlay
start_time = time.clock()
num_epochs = 1
iter = int(len(cube)/100)
proj = np.zeros((num_epochs,iter,3))
for epoch in range(num_epochs):
    theta = 0+0.1*time.clock()
    #z_rot = np.array([[math.cos(4.71238898038 ),math.sin(4.71238898038),0],
     #           [-math.sin(4.71238898038),math.cos(4.71238898038),0],
      #          [0,0,1]])
    z_rot = np.array([[int(1000*math.cos(theta)),int(1000*math.sin(theta)),0],
                     [int(1000*-math.sin(theta)),int(1000*math.cos(theta)),0],
                    [0, 0, 1]])
    x_rot = np.array([[1,0,0],
                  [0,math.cos(theta),-math.sin(theta)],
                  [0,math.sin(theta),math.cos(theta)]])

    y_rot = np.array([[math.cos(theta),0,math.sin(theta)],
                  [0,1,0],
                  [-math.sin(theta),0,math.cos(theta)]])


    cube_rot = np.zeros((iter,3))
    
    rot_operation_start = time.clock()
    
    for i in range(iter):
        rot_start = time.clock()
        
#         cube_rot[i,0],cube_rot[i,1],cube_rot[i,2] = overlay.mat_calc.mat_calc(int(cube[i,0]),int(cube[i,1]),int(cube[i,2]),int(z_rot[0,0]),int(z_rot[0,1]),int(z_rot[0,2]),int(z_rot[1,0]),int(z_rot[1,1]),int(z_rot[1,2]),int(z_rot[2,0]),int(z_rot[2,1]),int(z_rot[2,2]))
        cube_rot[i,0],cube_rot[i,1],cube_rot[i,2] = mat_calc(int(cube[i,0]),int(cube[i,1]),int(cube[i,2]),int(z_rot[0,0]),int(z_rot[0,1]),int(z_rot[0,2]),int(z_rot[1,0]),int(z_rot[1,1]),int(z_rot[1,2]),int(z_rot[2,0]),int(z_rot[2,1]),int(z_rot[2,2]))

    # Constrain +ve -ve
        if(cube_rot[i,0]>500000000):
            cube_rot[i,0] = -1*(4294967296 - cube_rot[i,0])
    
        if(cube_rot[i,1]>500000000):
            cube_rot[i,1] = -1*(4294967296 - cube_rot[i,1])
        #cube_rot[i,0],cube_rot[i,1],cube_rot[i,2] = mat_calc(cube[i,0],cube[i,1],cube[i,2],x_rot)
        #pass
        cube_rot[i,0] = cube_rot[i,0]/1000
        cube_rot[i,1] = cube_rot[i,1]/1000
        cube_rot[i,2] = cube_rot[i,2]/1000
        rot_end = time.clock()
        rot_elapsed = rot_end - rot_start
        print("Rotation: ",i, rot_elapsed)
    
    rot_operation_end = time.clock()
    rot_operation_elapsed = rot_operation_end - rot_operation_start
    print("Rotated",epoch, rot_operation_elapsed)
    
    cube_trans = cube_rot
    #cube_trans = cube
    
    trans_operation_start = time.clock()
    for i in range(iter):
        trans_start = time.clock()
        cube_trans[i,2] = cube[i,2] + 3
#         proj[epoch,i] = overlay.mat_calc.mat_calc(int(cube_trans[i,0]),int(cube_trans[i,1]),int(cube_trans[i,2]),int(matrix[0,0]),int(matrix[0,1]),int(matrix[0,2]),int(matrix[1,0]),int(matrix[1,1]),int(matrix[1,2]),int(matrix[2,0]),int(matrix[2,1]),int(matrix[2,2]))
        proj[epoch,i] = mat_calc(int(cube_trans[i,0]),int(cube_trans[i,1]),int(cube_trans[i,2]),int(matrix[0,0]),int(matrix[0,1]),int(matrix[0,2]),int(matrix[1,0]),int(matrix[1,1]),int(matrix[1,2]),int(matrix[2,0]),int(matrix[2,1]),int(matrix[2,2]))
    
    #Constrain +ve -ve here
    
        if(proj[epoch,i,0]>500000000):
            proj[epoch,i,0] = -1*(4294967296 - proj[epoch,i,0])
    
        if(proj[epoch,i,1]>500000000):
            proj[epoch,i,1] = -1*(4294967296 - proj[epoch,i,1])
    
        proj[epoch,i,0] = proj[epoch,i,0]/500000
        proj[epoch,i,1] = proj[epoch,i,1]/1000000
        proj[epoch,i,2] = proj[epoch,i,2]/1000000
        
        trans_end = time.clock()
        trans_elapsed = time.clock()
        print("Translated: ",i,trans_elapsed)
    
    trans_operation_end = time.clock()
    trans_operation_elapsed = trans_operation_end - trans_operation_start
    print("Translated",epoch, trans_operation_elapsed)
    
    adj_operation_start = time.clock()
    for i in range(iter):
        proj[epoch,i,0] = 10*(proj[epoch,i,0]+10)
        proj[epoch,i,1] = 20*(proj[epoch,i,1]+15)
    
    adj_operation_end = time.clock()
    adj_operation_elapsed = trans_operation_end - trans_operation_start
    print("Adjusted",epoch, adj_operation_elapsed)
    
np.save("proj.npy",proj)
np.save("cube.npy",cube)
  
end_time = time.clock()
elapsed_time = end_time-start_time
print("Using the PYNQ processing unit ")
print("Start time: ",start_time)
print("End time: ",end_time)
print("Elapsed time: ",elapsed_time)